<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py -O preprocess_sku.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-10-18 21:11:11--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  3.55MB/s    in 1.6s    

2022-10-18 21:11:13 (3.55 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-10-18 21:11:13--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6756 (6.6K) [text/plain]
Saving to: ‘preprocess_sku.py’

preprocess_sku.py   100%[===================>]

In [2]:
# ! pip install -q deepmatch

In [3]:
import pandas as pd
from ast import literal_eval
from preprocess_sku import gen_data_set, gen_model_input, gen_data_set_timesplit
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss

In [2]:
%load_ext autoreload
%autoreload 2

# Data

In [4]:
# data_path = "./"

# unames = ['user_id','gender','age','occupation','zip']
# user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
# rnames = ['user_id','sku_number','rating','timestamp']
# ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
# mnames = ['sku_number','title','genres']
# movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
# movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

# data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w
# print(data.shape)
# print(data.user_id.nunique())
# print(data.sku_number.nunique())
# data.head(2)

### Sales data

In [5]:
data = pd.read_csv('sample_data/mik_sales_sp_10052022.csv', index_col = False, 
                   parse_dates = ['trans_date','created_time']
                   )
# data['most_recent_purchased_sku'] = data['most_recent_purchased_sku'].apply(literal_eval)
# data['most_recent_view_sku'] = data['most_recent_view_sku'].apply(literal_eval)
# data['hist_sku_number'] = data['most_recent_purchased_sku'] + data['most_recent_view_sku']
data.shape

(23206, 14)

In [6]:
data['user_id'].nunique(), data['sku_number'].nunique()

(200, 8036)

In [7]:
data['trans_date'].max(),data['trans_date'].min(), data['trans_date'].max() - data['trans_date'].min()

(Timestamp('2022-10-02 00:00:00'),
 Timestamp('2022-04-02 00:00:00'),
 Timedelta('183 days 00:00:00'))

In [8]:
import datetime
# test_start_date = pd.Timestamp(datetime.datetime.today().date() - datetime.timedelta(6))
test_start_date = data['trans_date'].max() - datetime.timedelta(6)

test_start_date

Timestamp('2022-09-26 00:00:00')

In [9]:
data.columns

Index(['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'created_time', 'email_address', 'geo_zip',
       'most_frequent_device_class_general', 'total_order_value',
       'total_coupon_value', 'most_recent_purchased_sku',
       'most_recent_view_sku', 'category_path'],
      dtype='object')

In [10]:
data = data[['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'created_time', 'email_address', 'geo_zip','most_frequent_device_class_general','category_path']]

In [11]:
data['sku_number_org'] = data['sku_number']

### Item category mapping dat

In [10]:
df_item = pd.read_csv("sample_data/mik_item_07292022.csv", index_col=False)
df_item.shape

FileNotFoundError: ignored

In [11]:
df_item.head(2)

NameError: ignored

In [ ]:
data['most_recent_view_sku'].apply(lambda x: len(x)).describe()

# Feature Engineering

In [12]:
sparse_features = ['sku_number', 'category_path',
                   'user_id', 'geo_zip','most_frequent_device_class_general']

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


In [36]:
# view_item_set = list(set(data['most_recent_view_sku'].sum()))
# full_item_set = list(set(list(data['sku_number'].unique()) + view_item_set))
# len(full_item_set)

In [13]:
feature_max_idx = {}
for feature in sparse_features:
    # if feature == 'sku_number':      
    #   lbe_sku = LabelEncoder()
    #   lbe_sku.fit(full_item_set)
    #   data[feature] = lbe_sku.transform(data[feature]) + 1
    #   feature_max_idx[feature] = data[feature].max() + 1
    # else:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

In [14]:
feature_max_idx

{'sku_number': 8037,
 'category_path': 795,
 'user_id': 201,
 'geo_zip': 186,
 'most_frequent_device_class_general': 3}

In [15]:
data.head(2)

,data_source,user_id,sku_number,qty,trans_date,created_time,email_address,geo_zip,most_frequent_device_class_general,category_path,sku_number_org
0,ADOBE,67,4118,1,2022-07-10,2022-07-10 09:08:18,jhitzke@yahoo.com,119,1,173,10549096
1,ADOBE,67,749,1,2022-05-27,2022-05-27 12:36:24,jhitzke@yahoo.com,119,1,667,10185462


In [16]:
user_profile = data[['user_id', 'geo_zip', 'most_frequent_device_class_general']].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["sku_number"]].drop_duplicates('sku_number')
user_profile.shape, item_profile.shape

((200, 2), (8036, 1))

In [17]:
# user_item_list = data.groupby("user_id")['sku_number'].apply(list)
user_item_list = data.groupby("user_id")['sku_number'].apply(list)

user_item_list.head()

user_id
1    [2738, 5785, 4318, 6007, 5783, 2337, 4455, 444...
2    [3720, 1912, 3726, 3719, 3726, 3731, 1888, 370...
3    [6729, 6001, 6730, 6727, 6728, 3726, 6730, 672...
4    [3721, 3719, 3708, 4678, 4679, 3721, 3512, 370...
5    [4397, 4965, 4233, 1654, 4241, 374, 6251, 4963...
Name: sku_number, dtype: object

In [18]:
train_set, test_set = gen_data_set_timesplit(data, test_start_date, SEQ_LEN, NUMBER_NEG_SAMPLE)

100%|██████████| 200/200 [00:01<00:00, 120.23it/s]

7 7


In [ ]:
"# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

In [19]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess_sku.py:97: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((44840,), (44840,), (586,), (586,))

In [20]:
train_model_input['user_id'].shape

(44840,)

# Create Embeddings

In [21]:
from collections import Counter
train_counter = Counter(train_model_input['sku_number'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['sku_number'])]
len(train_counter), type(train_counter), len(item_count)

(8019, collections.Counter, 8037)

In [22]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

### Create Feature Embeddings



In [23]:
# item features
item_feature_columns = ['sku_number']
# User features
user_feature_columns = ['user_id', 'geo_zip', 'most_frequent_device_class_general',                        
                        'hist_sku_number',
                        'hist_category_path',
                        'hist_len'
                        ]


In [24]:
feature_max_idx

{'sku_number': 8037,
 'category_path': 795,
 'user_id': 201,
 'geo_zip': 186,
 'most_frequent_device_class_general': 3}

In [25]:
user_sparse_feature_columns = user_feature_columns[0:3]
user_sparse_feature_columns

['user_id', 'geo_zip', 'most_frequent_device_class_general']

In [26]:
user_seq_sparse_feature_columns = user_feature_columns[3:-1]
user_seq_sparse_feature_columns 

['hist_sku_number', 'hist_category_path']

In [27]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [28]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
   emb.trainable = True
   sparse_embedding[name] = emb

In [29]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

In [30]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fc9498511d0>,
 'geo_zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fc94c9685d0>,
 'most_frequent_device_class_general': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fc948eba850>,
 'hist_sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fc9bf488310>,
 'hist_category_path': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fc948ebac50>,
 'sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fc94cb98190>}

# Create User Inputs


## Create Inputs Tensor

In [31]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

In [32]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'geo_zip:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'most_frequent_device_class_general:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_sku_number:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_category_path:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len:0' shape=(None, 1) dtype=int32>]

In [33]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


In [34]:
item_inputs_list

[<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>]

## Embed Inputs

In [35]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

In [36]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

In [37]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

In [38]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

In [39]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_geo_zip/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_most_frequent_device_class_general/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_1/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

## Combine Embeded Input (User only)

In [40]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

In [41]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten/Reshape:0' shape=(None, 112) dtype=float32>

# Sepecify Model Layers

### User Layers

In [42]:
from tensorflow.keras import layers

In [43]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

In [44]:
# from deepctr.layers import DNN
from model_utils import DNN

In [45]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



In [46]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

### Item Layer

In [47]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


<tf.Tensor 'lambda_1/l2_normalize:0' shape=(8037, 32) dtype=float32>

In [48]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(8037, 32)

In [49]:
# .eval(session=K.get_session())

### Output Layer

In [50]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer/ExpandDims:0' shape=(None, 1) dtype=float32>

In [51]:
# from model_utils import SampledSoftmaxLayer
# sf = SampledSoftmaxLayer()

In [52]:
# from tensorflow.keras.layers import Softmax
# sf2 = Softmax()
# import numpy as np
# x = np.asarray([[1., 2., 1.]])
# x
# sf2(x).eval(session=K.get_session())

In [53]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_1/l2_normalize:0' shape=(8037, 32) dtype=float32>

In [54]:
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

In [55]:
item_features[item_feature_name]

<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>

## Build Model

In [56]:
from tensorflow.python.keras.models import Model

In [57]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


# Model Compile and Training

In [58]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

In [59]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

Train on 44840 samples
Epoch 1/20
44840/44840 [==============================] - 4s 99us/sample - loss: 4.8563
Epoch 2/20
44840/44840 [==============================] - 4s 86us/sample - loss: 4.7753
Epoch 3/20
44840/44840 [==============================] - 4s 84us/sample - loss: 4.7012
Epoch 4/20
44840/44840 [==============================] - 4s 85us/sample - loss: 4.6048
Epoch 5/20
44840/44840 [==============================] - 4s 83us/sample - loss: 4.5576
Epoch 6/20
44840/44840 [==============================] - 4s 84us/sample - loss: 4.5283
Epoch 7/20
44840/44840 [==============================] - 4s 87us/sample - loss: 4.4885
Epoch 8/20
44840/44840 [==============================] - 4s 87us/sample - loss: 4.4774
Epoch 9/20
44840/44840 [==============================] - 4s 86us/sample - loss: 4.4545
Epoch 10/20
44840/44840 [==============================] - 4s 86us/sample - loss: 4.4389
Epoch 11/20
44840/44840 [==============================] - 4s 85us/sample - loss: 4.4267
Epoch 1

In [60]:
res = model.predict(test_model_input)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [61]:
res.shape

(586, 1)

# Predict: Generate Item and User Embeddings

In [62]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"sku_number": item_profile['sku_number'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(586, 32)
(8036, 32)


In [63]:
prob_matrix = user_embs.dot(item_embs.T)
prob_matrix.shape

(586, 8036)

# Recommendation

In [64]:
! pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 4.2 MB/s 


In [65]:
import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

In [66]:
def recall_N(y_true, y_pred, N=50):
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / len(y_true)

In [67]:
test_true_label = {line[0]:[line[1]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0
pred_label = {}
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['sku_number'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        pred_label[uid] = pred
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))

586it [00:00, 3071.83it/s]


recall 0.09556313993174062
hit rate 0.09556313993174062


In [68]:
len(pred_label[160])

50

# Validation

In [69]:
df_train = pd.DataFrame(train_set)

In [81]:
test_user_model_input['user_id'][0:10]

array([ 78,  58, 158,  79,  20,  89,  78, 164,  64,  51])

In [85]:
user_idx = 0
uid = 78

In [86]:
D[user_idx]

array([0.9612281 , 0.9417473 , 0.9397803 , 0.93913186, 0.9276499 ,
       0.9266509 , 0.92628735, 0.9223522 , 0.9212135 , 0.91644645,
       0.91586614, 0.91485965, 0.913774  , 0.91183794, 0.9116538 ,
       0.901401  , 0.8980908 , 0.8811122 , 0.87330663, 0.8731355 ,
       0.8720013 , 0.87110955, 0.86865544, 0.86742294, 0.86515105,
       0.8645307 , 0.8638684 , 0.85951537, 0.8585889 , 0.85515654,
       0.8491323 , 0.84098256, 0.83728874, 0.8360081 , 0.82600945,
       0.8246573 , 0.8219001 , 0.82165897, 0.8215759 , 0.81991285,
       0.8181087 , 0.8176801 , 0.81616014, 0.81459105, 0.81248516,
       0.80685866, 0.8052764 , 0.8040533 , 0.8011446 , 0.7972148 ],
      dtype=float32)

In [87]:
sorted_index = prob_matrix.argsort()[:,-50:]
-np.sort(-prob_matrix[user_idx][sorted_index[user_idx]])

array([0.9612281 , 0.9417473 , 0.9397803 , 0.93913186, 0.9276499 ,
       0.9266509 , 0.92628735, 0.9223522 , 0.9212135 , 0.91644645,
       0.91586614, 0.91485965, 0.913774  , 0.91183794, 0.9116538 ,
       0.901401  , 0.8980908 , 0.8811122 , 0.87330663, 0.8731355 ,
       0.8720013 , 0.87110955, 0.86865544, 0.86742294, 0.86515105,
       0.8645307 , 0.8638684 , 0.85951537, 0.8585889 , 0.85515654,
       0.8491323 , 0.84098256, 0.83728874, 0.8360081 , 0.82600945,
       0.8246573 , 0.8219001 , 0.82165897, 0.8215759 , 0.81991285,
       0.8181087 , 0.8176801 , 0.81616014, 0.81459105, 0.81248516,
       0.80685866, 0.8052764 , 0.8040533 , 0.8011446 , 0.7972148 ],
      dtype=float32)

In [88]:
I[user_idx]

array([2726,  999, 1064, 1843, 1851, 1074,  446,  541, 2732, 1072,  881,
       1071, 1854, 1066, 3006, 1849, 1068, 1070, 4488, 1003, 3745, 1848,
       1001,  995,  928, 5080, 1842, 1738, 2733, 1006, 3007,  553, 1004,
       6938, 1442, 4293, 5016, 4012, 2769, 6593, 7413, 7276, 1445,  549,
        597,  993, 4512, 1850, 1508, 7144])

In [89]:
(sorted_index[user_idx])[::-1]

array([2726,  999, 1064, 1843, 1851, 1074,  446,  541, 2732, 1072,  881,
       1071, 1854, 1066, 3006, 1849, 1068, 1070, 4488, 1003, 3745, 1848,
       1001,  995,  928, 5080, 1842, 1738, 2733, 1006, 3007,  553, 1004,
       6938, 1442, 4293, 5016, 4012, 2769, 6593, 7413, 7276, 1445,  549,
        597,  993, 4512, 1850, 1508, 7144])

In [90]:
np.array([item_profile['sku_number'].values[x] for x in I[user_idx]])

array([4711, 4715, 5050, 5049, 4710, 5051, 3130, 5304, 4716, 5043, 3122,
       5061, 5045, 5041, 6322, 5052, 5057, 4396, 5058, 4721, 5047, 5053,
       5062, 5048,  776, 4719, 5044, 5288, 4724, 4720, 5056, 5088, 5087,
       3753, 1346, 5296,  134, 7576, 2666, 7529, 1017, 5060, 5287, 1339,
       1344, 4395, 1340, 5055,  742, 5103])

In [91]:
len(set(df_train[df_train[0]==uid][1].unique()) & set([item_profile['sku_number'].values[x] for x in I[user_idx]]))

17

In [92]:
len(set(df_train[df_train[0]==uid][1].unique()) & set(sorted_index[user_idx]))

1

In [93]:
item_profile['sku_number']

0        4118
1         749
2         753
3        7976
4        4599
         ... 
23177     674
23181    1700
23182    3286
23193    5297
23195    5298
Name: sku_number, Length: 8036, dtype: int64

In [94]:
all_item_model_input

{'sku_number': array([4118,  749,  753, ..., 3286, 5297, 5298])}